# Verifying the Accuracy of iGEM Toronto's Flux Scanning Based on Enforced objective Flux (FSEOF) Implementation

As part of their 2023 project, the Dry Lab sub-team of the University of Toronto's iGEM team implemented the Flux Scanning Based on Enforced objective Flux (FSEOF) method, based on the original FSEOF framework presented in the paper by Choi et al., titled: *In silico identification of gene amplification targets for improvement of lycopene production*. FSEOF is used to identify candidate genes that can be overexpressed in order to achieve a metabolic engineering objective.

<in order to validate our implementation, we decided to replicate so and so experiment (from the paper) to see if we got the same candidate genes for improved lycopene biosynthesis



*Implementation of FSEOF on lycopene-producing E. coli*
Choi et. al 2010: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2869140/

In [1]:
import sys
sys.path.append("..")

## Construction of the lycopene-producing *E. coli*

The parental *E. coli* strain that was used in the paper to experimentally validate the targets identified by FSEOF (paper's FSEOF) is a "recombinant E. coli DH5 alpha strain... which contains the *Erwinia uredovora crtEIB* (lycopene biosynthesis) genes and the *E. coli dxs* gene." (Choi et al. 2010)

< with the exception of dxs (they added it for the purposes of overexpression, but we cant simulate that), since it was already present in the model)>

The genome scale metabolic model used in the paper for the *in-silico* metabolic modeling and analysis of the recombinant E. coli DH5 alpha strain (using MetaFluxNet) was EcoMBEL979, which was expanded to include the lycopene biosynthetic pathway genes (crtEIB).

Instead of using the expanded EcoMBEL979 model for the *in-silico* metabolic modeling and analysis of the recombinant E. coli DH5 alpha strain, this example uses the already available BiGG model for E. coli DH5 alpha (BiGG ID: iEC1368_DH5a), and expands it to include the necessary lycopene biosynthetic pathway genes, crtE, crtB and crtI.

In addition to the lycopene biosynthetic genes, a lycopene demand reaction was added. [Should I explain why here?]
Ref: https://cnls.lanl.gov/external/qbio2018/Slides/FBA%202/qBio-FBA-lab-slides.pdf

Reference for the exact genes (and their reactions) added can be found in choi et. al supplementary file 3, page 11.





In [2]:
import examples
import fseof_additions
import fseof_media
import dh5_alpha

#load model
model = dh5_alpha.ecoli_dh5a()

#add necessary genes (lycopene pathways)
fseof_additions.add_crtE(model)
fseof_additions.add_crtB(model)
fseof_additions.add_crtI(model)
fseof_additions.add_lyco_dem(model)







## Media Setup

Next, the media must be set. [Explain media setup]


In [ ]:
#set LB media
fseof_media.simulate_LB_media(model)

In [3]:
#check objective to see if matches the one presented in paper

#max theoretical lycopene flux
model.objective = "LYCOdem"
solution = model.optimize()
solution.objective_value


0.9657492354740086

Now, we will run our implementation of FSEOF on the model created above.

In [4]:
#run fseof to cross-check with choi et al. paper
import cobra_fseof
results = cobra_fseof.fseof(model, 10, "LYCOdem", "BIOMASS_Ec_iJO1366_core_53p95M", "max")
print(results.scan)

FSEOF; Running FVA: 100%|██████████| 10/10 [03:59<00:00, 23.97s/it]

                              target  vmin      vmax  flux0     flux1  \
23CUMPtex                      False   0.0  0.000000    0.0  0.000000   
23DAPPAt2pp                    False   0.0  0.000000    0.0  0.000000   
23DAPPAtex                     False   0.0  0.000000    0.0  0.000000   
BIOMASS_Ec_iJO1366_WT_53p95M   False   0.0  0.000000    0.0  0.000000   
23PDE2pp                       False   0.0  0.000000    0.0  0.000000   
...                              ...   ...       ...    ...       ...   
UAG4Ei                         False   0.0  0.000000    0.0  0.000000   
UGCIAMH                        False   0.0  0.000000    0.0  0.000000   
ZCRTE                           True   0.0  1.738349    0.0  0.193150   
ZCRTB                           True   0.0  0.869174    0.0  0.096575   
ZCRTI                           True   0.0  0.869174    0.0  0.096575   

                                flux2     flux3     flux4     flux5     flux6  \
23CUMPtex                     0.00000  0.0

In [5]:
#print list of reactions that our fseof identiified
targets = zip(results.scan.index, results.scan.target)
targets = dict(targets)
lst = []
for target in targets:
    if targets[target]:
        lst.append(target)

print(lst)





['AKGDH', 'CDPMEK', 'CYTK1', 'DMATT', 'DXPRIi', 'DXPS', 'FBA3', 'FE3Ri', 'FESD1s', 'FESR', 'GRTT', 'IPDDI', 'IPDPS', 'MECDPDH5', 'MECDPS', 'MEPCT', 'MOX', 'PIt2rpp', 'PItex', 'PFK_3', 'NDPK3', 'PGI', 'PGMT', 'POR5', 'PYK', 'RPE', 'TKT1', 'TKT2', 'TPI', 'MECDPDH2', 'ZCRTE', 'ZCRTB', 'ZCRTI']


In [6]:
# choi et al. 2010 Supplemental Table 4A (All reactions that were identified by their fseof)

#TCA cycle reactions
#'FUM_rxn' not in model, so replaced it with 'FUM'
tableFourTCA = ['ACONT', 'CS', 'FUM', 'ICDHyr', 'MDH', 'SUCDli', 'SUCOAS', 'AKGDH', 'SUCD4']


# 'MECDPDH5' from table 4A, is replaced with 'MECDPDH2' and 'MECDPDH5' here
tableFourLyc = ['CDPMEK', 'DMATT', 'DXPRIi', 'DXPS','GRTT', 'IPDDI', 'IPDPS', 'MECDPDH5', 'MECDPDH2', 'MECDPS', 'MEPCT', 'ZCRTE', 'ZCRTB', 'ZCRTI']


#TODO: PFK_3==PFK (but theres also PFK in model...)
# FBA is replaced with 'FBA3' http://bigg.ucsd.edu/models/iEC1368_DH5a/genes/locus_2187
tableFourGlyc = ['FBA3', 'PFK_3', 'PGI', 'TPI']


#disclaimer: 'CACTP' is replaced by 'CRNabcpp' (since model did not have CACTP rxn)
#CAITP is replaced by CRNt8pp (since model did not have CAITP rxn)
tableFourOther = ['ADK1', 'ADK4', 'CYTK1', 'CRNabcpp', 'CRNt8pp', 'PPA', 'CO2t', 'H2Ot', 'PIt2rpp']




In [9]:
#Of the genes in supp file 4, how many was our algorithm able to identify?

tableFourMegalst = tableFourTCA + tableFourLyc + tableFourGlyc + tableFourOther
print("Total number of reactions that the choi et. al 2010 paper identified: " + str(len(tableFourMegalst)))

count = 0
for item in tableFourMegalst:
    if item in lst:
        count += 1

print("How many out of the total number of reactions that the choi et. al paper identified, did our FSEOF identify: " + str(count) + "/" + str(len(tableFourMegalst)))
print("Above value as a percentage: " + str(count/len(tableFourMegalst)))


count1 = 0
for item in tableFourLyc:
    if item in lst:
        count1 += 1

print("What percentage of the Lycopene biosynthetic pathways that choi et. al identified, did our fseof identify: " + str(count1/len(tableFourLyc)))




Total number of reactions that the choi et. al 2010 paper identified: 36
How many out of the total number of reactions that the choi et. al paper identified, did our FSEOF identify: 21/36
Above value as a percentage: 0.5833333333333334
What percentage of the Lycopene biosynthetic pathways that choi et. al identified, did our fseof identify: 1.0
